In [ ]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb

from config.config import (
    FE_PATH,
    MES_TEST_FINAL,
    MODELOS_PATH,
    RESULTADOS_PREDICCION_PATH,
    NOMBRE_EXPERIMENTO,
)
from src.data_load_preparation import cargar_datos, preparar_test_final


In [ ]:
# Cargar dataset de features
data = cargar_datos(FE_PATH)

# Test final según config (MES_TEST_FINAL = [202108])
X_test, clientes_test = preparar_test_final(data)

# Por si querés también el subset completo de 202108 con todas las columnas:
data_test = data[data["foto_mes"].isin(MES_TEST_FINAL)].copy()

len(X_test), len(clientes_test), len(data_test)


In [ ]:
import glob

model_pattern = os.path.join(MODELOS_PATH, f"{NOMBRE_EXPERIMENTO}_seed*_final.txt")
model_files = sorted(glob.glob(model_pattern))

print("Modelos encontrados:")
for f in model_files:
    print(" -", os.path.basename(f))


In [ ]:
all_preds = []

for model_path in model_files:
    print(f"📂 Cargando modelo: {os.path.basename(model_path)}")
    model = lgb.Booster(model_file=model_path)
    y_pred = model.predict(X_test)
    all_preds.append(y_pred)

all_preds = np.vstack(all_preds)   # shape = (n_modelos, n_clientes)
prob_ensemble = all_preds.mean(axis=0)

prob_ensemble.shape


In [ ]:
UMBRAL_ENSEMBLE = 0.044330  # del log

pred_binaria = (prob_ensemble >= UMBRAL_ENSEMBLE).astype(int)
N_enviados = int(pred_binaria.sum())

N_enviados, N_enviados / len(pred_binaria) * 100


In [ ]:
from datetime import datetime

os.makedirs(RESULTADOS_PREDICCION_PATH, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
filename = (
    f"{NOMBRE_EXPERIMENTO}_ensemble_desde_modelos_"
    f"U{UMBRAL_ENSEMBLE:.6f}_N{N_enviados}_{timestamp}.csv"
)
output_path = os.path.join(RESULTADOS_PREDICCION_PATH, filename)

submission_202108 = pd.DataFrame({
    "numero_de_cliente": clientes_test,
    "Predicted": pred_binaria
})

submission_202108.to_csv(output_path, index=False)

output_path
